Здесь реализовал пример использования модели для инференса. В задании это было довольно расплывчато, поэтому остановился на такой базовой демонстрации работы. Конечно, это должен быть еще и готовый rest сервис.

In [1]:
from transformers import AutoTokenizer
from contrastive.model import DistilBertForSequenceClassification
import torch

THRESHOLD = -3951.605712890625
TOKENIZER_PATH = 'runs/BERT_contrastive/checkpoint-7190/'
MODEL_NAME = 'runs/BERT_contrastive/checkpoint-7190/model.pt'
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, use_fast=True, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(MODEL_NAME, map_location=device)

In [3]:
def infer(input_text):
    sample = tokenizer(input_text, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
    model.eval()
    with torch.no_grad():
        ans = model(**sample.to(device))
    
    pooled = ans[2]
    maha_score = []
    for c in model.all_classes:
        centered_pooled = pooled - model.class_mean[c].unsqueeze(0)
        ms = torch.diag(centered_pooled @ model.class_var @ centered_pooled.t())
        maha_score.append(ms)
    maha_score = torch.stack(maha_score, dim=-1)
    maha_score = maha_score.min(-1)[0]
    maha_score = -maha_score
    
    if maha_score < THRESHOLD:
        print(maha_score)
        return 'ood'
    else:
        ans = torch.argmax(ans[0]).item()
        return ans

In [4]:
input_text = 'Фильм ориентирован на маленьких детей не старше 10 лет. Смешного в фильме совсем мало. В графе жанр нужно было написать просто детская сказка. Стиллер играет неплохо, только в сценарии, как я уже отмечал, почти нет смешных сцен.'

In [5]:
infer(input_text)

tensor([-11751.3408], device='cuda:0')


'ood'

In [6]:
input_text = 'убрать будильник который я только что поставил'

In [7]:
infer(input_text)

52

In [8]:
input_text = 'включи мой рок плейлист'
infer(input_text)

45

In [9]:
input_text = 'какая погода'
infer(input_text)

13

In [10]:
input_text = 'расскажи мне смешную шутку'
infer(input_text)

25

In [11]:
input_text = 'Абсолютно не понравился. Фильм о великовозрастных жеребцах-халявщиках. Так же на халяву они понаходили своих прекрасных богатых невест. Сюжет предсказуемый с первой до последней минуты, юмор традиционный для таких комедий и несмешной. Одно хорошо: к фильму напрашивается фраза Любовь спасёт мир.'
infer(input_text)

tensor([-14325.5186], device='cuda:0')


'ood'

In [12]:
input_text = 'Абсолютно не понравился. Фильм о великовозрастных жеребцах-халявщиках. Так же на халяву они понаходили своих прекрасных богатых невест. Сюжет предсказуемый с первой до последней минуты, юмор традиционный для таких комедий и несмешной. Одно хорошо: к фильму напрашивается фраза Любовь спасёт мир.'
infer(input_text.lower())

tensor([-12721.0625], device='cuda:0')


'ood'